In [7]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['CNS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('central nervous system', 4400),
 ('coagulase negative staphylococci', 37),
 ('central', 20),
 ('congenital nephrotic syndrome', 11),
 ('central nerve system', 11),
 ('conserved noncoding sequences', 7),
 ('coagulase negative staphylococcus', 5),
 ('non coding sequences', 5),
 ('crigler najjar syndrome', 3),
 ('carbon nanospheres', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [8]:
d = load_disambiguator('CNS')

In [9]:
results = [d.grounding_dict['CNS'], d.names, d.pos_labels]

In [10]:
results

[{'carbon nanospheres': 'ungrounded',
  'cavernosal nerve stimulation': 'ungrounded',
  'celestial navigation system': 'ungrounded',
  'cen tral nervous system': 'MESH:D002490',
  'central ner vous system': 'MESH:D002490',
  'central nerve system': 'MESH:D002490',
  'central nervous': 'MESH:D002490',
  'central nervous sys tem': 'MESH:D002490',
  'central nervous system': 'MESH:D002490',
  'central nervous system infection': 'ungrounded',
  'central nervous sytem': 'ungrounded',
  'central neural system': 'MESH:D002490',
  'clinical nurse specialist': 'ungrounded',
  'cns': 'ungrounded',
  'coagulase negative staphylococcal': 'coagulase_negative_staphylococci',
  'coagulase negative staphylococci': 'coagulase_negative_staphylococci',
  'coagulase negative staphylococcus': 'coagulase_negative_staphylococci',
  'coagulase negative staphylococcus spp': 'coagulase_negative_staphylococci',
  'congenital nephrotic syndrome': 'NCIT:C35337',
  'congenital ns': 'NCIT:C35337',
  'conserved nonco

In [12]:
grounding_map, names, pos_labels = [{'carbon nanospheres': 'ungrounded',
  'cavernosal nerve stimulation': 'ungrounded',
  'celestial navigation system': 'ungrounded',
  'cen tral nervous system': 'MESH:D002490',
  'central ner vous system': 'MESH:D002490',
  'central nerve system': 'MESH:D002490',
  'central nervous': 'MESH:D002490',
  'central nervous sys tem': 'MESH:D002490',
  'central nervous system': 'MESH:D002490',
  'central nervous system infection': 'ungrounded',
  'central nervous sytem': 'ungrounded',
  'central neural system': 'MESH:D002490',
  'clinical nurse specialist': 'ungrounded',
  'cns': 'ungrounded',
  'coagulase negative staphylococcal': 'coagulase_negative_staphylococci',
  'coagulase negative staphylococci': 'coagulase_negative_staphylococci',
  'coagulase negative staphylococcus': 'coagulase_negative_staphylococci',
  'coagulase negative staphylococcus spp': 'coagulase_negative_staphylococci',
  'congenital nephrotic syndrome': 'NCIT:C35337',
  'congenital ns': 'NCIT:C35337',
  'conserved noncoding sequence': 'conserved_non_coding_sequence',
  'crigler najjar syndrome': 'ungrounded',
  'non coding dna sequence': 'conserved_non_coding_sequence',
  'non coding sequence': 'conserved_non_coding_sequence'},
 {'MESH:D002490': 'Central Nervous System',
  'coagulase_negative_staphylococci': 'Coagulase Negative Staphylococci',
  'NCIT:C35337': 'Congenital Nephrotic Syndrome',
  'conserved_non_coding_sequence': 'Conserved Non-Coding Sequence'},
 ['MESH:D002490', 'NCIT:C35337']]

In [13]:
excluded_longforms = ['cns']

In [14]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [15]:
additional_entities = {}

In [16]:
unambiguous_agent_texts = {}

In [17]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [17]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [18]:
names.update({key: value[0] for key, value in additional_entities.values()})
pos_labels = list(set(pos_labels) | additional_entities.keys())

In [18]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 00:24:29] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 00:25:23] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9947541292877391 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [19]:
classifier.stats

{'label_distribution': {'MESH:D002490': 3543,
  'coagulase_negative_staphylococci': 34,
  'NCIT:C35337': 9,
  'ungrounded': 5},
 'f1': {'mean': 0.994754, 'std': 0.001634},
 'precision': {'mean': 0.992433, 'std': 0.003012},
 'recall': {'mean': 0.997184, 'std': 0.000891},
 'MESH:D002490': {'f1': {'mean': 0.996341, 'std': 0.001121},
  'pr': {'mean': 0.998871, 'std': 0.000564},
  'rc': {'mean': 0.993827, 'std': 0.002265}},
 'NCIT:C35337': {'f1': {'mean': 0.333333, 'std': 0.421637},
  'pr': {'mean': 0.3, 'std': 0.4},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'ungrounded': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'coagulase_negative_staphylococci': {'f1': {'mean': 0.736044,
   'std': 0.188069},
  'pr': {'mean': 0.685714, 'std': 0.245781},
  'rc': {'mean': 0.83619, 'std': 0.10748}}}

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for CNS

Produces the disambiguations:
	Central Nervous System*	MESH:D002490
	Coagulase Negative Staphylococci	coagulase_negative_staphylococci
	Congenital Nephrotic Syndrome*	NCIT:C35337
	Conserved Non-Coding Sequence	conserved_non_coding_sequence

Class level metrics:
--------------------
Grounding                       	Count	F1     
          Central Nervous System*	3543	0.99634
Coagulase Negative Staphylococci	  34	0.73604
   Congenital Nephrotic Syndrome*	   9	0.33333
                      Ungrounded	   5	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.99475
	Precision:	0.99243
	Recall:		0.99718

* Positive labels
See Docstring for explanation



In [23]:
model_to_s3(disamb)